In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'f:\\Project_MultitaskModel'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    

In [4]:
from src.Project_MultitaskModel.constants import *
from src.Project_MultitaskModel.utils.common import read_yaml, create_directories

In [5]:
class ConfigureManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [6]:
import os
import zipfile
import boto3
from urllib.parse import urlparse
from pathlib import Path
# Import entity và logger của bạn ở đây...

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Tải file ZIP từ S3 về máy local
        """
        if not os.path.exists(self.config.local_data_file):
            # Parse URL: s3://bucket-name/path/to/data.zip
            parsed_url = urlparse(self.config.source_URL)
            bucket_name = parsed_url.netloc
            s3_key = parsed_url.path.lstrip('/')

            print(f"Downloading {s3_key} from bucket {bucket_name}...")

            try:
                s3 = boto3.client('s3')
                s3.download_file(bucket_name, s3_key, str(self.config.local_data_file))
                print(f"✅ Downloaded to: {self.config.local_data_file}")
            except Exception as e:
                print(f"❌ Error downloading: {e}")
                raise e
        else:
            print(f"File already exists: {self.config.local_data_file}")

    def extract_zip_file(self):
        """
        Giải nén file zip vào thư mục unzip_dir
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        print(f"Extracting zip file to: {unzip_path}...")
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            print("✅ Extracted successfully!")
        except Exception as e:
            print(f"❌ Error extracting: {e}")
            raise e

In [7]:
try:
    config = ConfigureManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # 1. Tải file zip
    data_ingestion.download_file()
    
    # 2. Giải nén
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2025-12-18 13:06:01,728: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2025-12-18 13:06:01,730: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-18 13:06:01,731: INFO: common: created directory at: artifacts]
[2025-12-18 13:06:01,732: INFO: common: created directory at: artifacts/data_ingestion]
File already exists: artifacts\data_ingestion\data.zip
Extracting zip file to: artifacts\data_ingestion...
✅ Extracted successfully!
